## 0 Import depencies

In [2]:
import mediapipe as mp
import cv2

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [ ]:
mp_holistic.HAND_CONNECTIONS

## 1 Make detections

In [99]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor feed
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        # Make detections
        results = holistic.process(image)

        image.flags.writeable=True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION,
                              mp_drawing.DrawingSpec(color=(255,0,0),thickness=1,circle_radius=1),
                              mp_drawing.DrawingSpec(color=(240,0,0),thickness=1,circle_radius=1))

        # 2. Left hand landmarks
        mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(255,0,0),thickness=2,circle_radius=2),
                              mp_drawing.DrawingSpec(color=(240,0,0),thickness=2,circle_radius=2))

        # 3. Right hand landmarks
        mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(255,0,0),thickness=2,circle_radius=2),
                              mp_drawing.DrawingSpec(color=(240,0,0),thickness=2,circle_radius=2))

        # 4. Pose detection
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(255,0,0),thickness=2,circle_radius=2),
                              mp_drawing.DrawingSpec(color=(240,0,0),thickness=2,circle_radius=2))

        cv2.imshow("Holistic model detection",image)

        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
cap.release()
cv2.destroyAllWindows()            

## 2 Capture landmarks and Export to CSV

In [29]:
import os,sys
import numpy as np

In [91]:
num_coords = len(results.pose_landmarks.landmark) + len(results.face_landmarks.landmark)
num_coords

501

In [92]:
landmarks = ['class']
for val in range(1,num_coords+1):
    landmarks+=[f"x{val}, y{val}, z{val}, v{val}"] 

In [93]:
import csv
os.makedirs('data',exist_ok=True)
with open('data/coords.csv',mode="w",newline="") as f:
    csv_writer = csv.writer(f,delimiter=",",quotechar="'",quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [124]:
class_name = "Happy"

In [125]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor feed
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        # Make detections
        results = holistic.process(image)

        image.flags.writeable=True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION,
                              mp_drawing.DrawingSpec(color=(255,0,0),thickness=1,circle_radius=1),
                              mp_drawing.DrawingSpec(color=(240,0,0),thickness=1,circle_radius=1))

        # 2. Left hand landmarks
        mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(255,0,0),thickness=2,circle_radius=2),
                              mp_drawing.DrawingSpec(color=(240,0,0),thickness=2,circle_radius=2))

        # 3. Right hand landmarks
        mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(255,0,0),thickness=2,circle_radius=2),
                              mp_drawing.DrawingSpec(color=(240,0,0),thickness=2,circle_radius=2))

        # 4. Pose detection
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(255,0,0),thickness=2,circle_radius=2),
                              mp_drawing.DrawingSpec(color=(240,0,0),thickness=2,circle_radius=2))
        
        # Export coordinates
        try:
            # Extract pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in pose]).flatten())

            # Extract face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in face]).flatten())

            # Concat rows
            row = pose_row+face_row

            # Insert row
            row.insert(0,class_name)

            # Export to csv
            with open('data/coords.csv',mode='a',newline='') as f:
                csv_writer = csv.writer(f,delimiter=",",quotechar="'",quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)

        except Exception as e:
            print(e)

        cv2.imshow("Holistic model detection",image)

        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
cap.release()
cv2.destroyAllWindows()            

## 3 Train custom model using scikit-learn

In [126]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression,RidgeClassifier
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier

### 3.1 Read in collected data and process

In [127]:
df = pd.read_csv('data/coords.csv')
df.sample(5)

,Victorius,0.4979402422904968,0.4681210517883301,-0.8602403402328491,0.9999120235443115,0.526459813117981,0.4148291349411011,-0.8045721054077148,0.9998873472213745,0.5431056022644043,...,-0.002525888616219163,0.0.465,0.5705472826957703,0.41139546036720276,0.012656714767217636,0.0.466,0.5750494003295898,0.4069294035434723,0.0130736930295825,0.0.467
139,Victorius,0.524360,0.502162,-0.562991,0.999988,0.556463,0.454222,-0.486707,0.999985,0.576274,...,-0.005465,0.0,0.580852,0.447459,-0.008492,0.0,0.585508,0.442914,-0.009618,0.0
930,Happy,0.382764,0.522209,-1.116260,0.999971,0.425730,0.431091,-1.048760,0.999918,0.452885,...,-0.011770,0.0,0.477971,0.420898,0.012765,0.0,0.484933,0.410403,0.013926,0.0
996,Happy,0.903546,0.586754,-1.226071,0.998469,0.930665,0.515564,-1.141603,0.996286,0.945367,...,-0.001712,0.0,1.026155,0.548450,0.036842,0.0,1.036082,0.543244,0.038877,0.0
454,Sad,0.392915,0.676347,-0.610584,0.998945,0.404959,0.617388,-0.609081,0.997950,0.417376,...,-0.011673,0.0,0.439795,0.591383,-0.009302,0.0,0.442105,0.586325,-0.010181,0.0
225,Victorius,0.549120,0.470696,-0.853573,0.999891,0.588575,0.399984,-0.779457,0.999805,0.613439,...,-0.009587,0.0,0.628756,0.377649,0.011414,0.0,0.633639,0.372683,0.011802,0.0


In [128]:
X=df.iloc[:,1:]
y=df.iloc[:,0]

In [129]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

### 3.2 Train machine learning classification model

In [130]:
pipelines = {
    "lr":make_pipeline(StandardScaler(),LogisticRegression()),
    "rc":make_pipeline(StandardScaler(),RidgeClassifier()),
    "rf":make_pipeline(StandardScaler(),RandomForestClassifier()),
    "gb":make_pipeline(StandardScaler(),GradientBoostingClassifier())
}

In [131]:
list(pipelines.values())[0]

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('logisticregression', LogisticRegression())])

In [132]:
fit_models = {}
for algo,pipeline in pipelines.items():
    model = pipeline.fit(X_train,y_train)
    fit_models[algo] = model

In [133]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

In [134]:
fit_models['lr'].predict(X_test)

array(['Sad', 'Sad', 'Happy', 'Happy', 'Victorius', 'Sad', 'Victorius',
       'Sad', 'Victorius', 'Sad', 'Happy', 'Happy', 'Victorius', 'Sad',
       'Happy', 'Happy', 'Sad', 'Sad', 'Happy', 'Happy', 'Victorius',
       'Happy', 'Sad', 'Happy', 'Sad', 'Victorius', 'Victorius',
       'Victorius', 'Victorius', 'Victorius', 'Victorius', 'Happy', 'Sad',
       'Sad', 'Victorius', 'Happy', 'Sad', 'Victorius', 'Sad',
       'Victorius', 'Happy', 'Happy', 'Sad', 'Sad', 'Victorius',
       'Victorius', 'Victorius', 'Sad', 'Happy', 'Happy', 'Happy',
       'Victorius', 'Happy', 'Happy', 'Sad', 'Sad', 'Victorius',
       'Victorius', 'Sad', 'Happy', 'Happy', 'Happy', 'Sad', 'Happy',
       'Victorius', 'Sad', 'Sad', 'Happy', 'Sad', 'Happy', 'Sad', 'Happy',
       'Sad', 'Sad', 'Sad', 'Victorius', 'Happy', 'Victorius', 'Happy',
       'Sad', 'Victorius', 'Happy', 'Sad', 'Victorius', 'Sad',
       'Victorius', 'Sad', 'Happy', 'Victorius', 'Victorius', 'Victorius',
       'Sad', 'Victorius', 'Vic

### 3.3 Evaluate and serialize model

In [2]:
from sklearn.metrics import accuracy_score

In [136]:
for algo,model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo,accuracy_score(y_test,yhat))

lr 1.0
rc 1.0
rf 0.9968253968253968
gb 0.9968253968253968


In [137]:
sum((fit_models['rf'].predict(X_test)==y_test)==True)

314

In [138]:
import os
os.makedirs('model',exist_ok=True)
with open('model/pose_detection.pkl','wb') as f:
    pickle.dump(fit_models['rf'],f)

## 4 Make detections with model

In [3]:
import pickle
with open('model/pose_detection.pkl','rb') as f:
    model = pickle.load(f)

### Workflow for detections
1. Detect landmarks
2. Predict pose (class) based on landmark coordinates
3. Render landmarks and body language pose using opencv

In [7]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [5]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [10]:
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor feed
        image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        # Make detections
        results = holistic.process(image)

        image.flags.writeable=True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

        # 1. Draw face landmarks
        mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION,
                              mp_drawing.DrawingSpec(color=(255,0,0),thickness=1,circle_radius=1),
                              mp_drawing.DrawingSpec(color=(240,0,0),thickness=1,circle_radius=1))

        # 2. Left hand landmarks
        mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(255,0,0),thickness=2,circle_radius=2),
                              mp_drawing.DrawingSpec(color=(240,0,0),thickness=2,circle_radius=2))

        # 3. Right hand landmarks
        mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(255,0,0),thickness=2,circle_radius=2),
                              mp_drawing.DrawingSpec(color=(240,0,0),thickness=2,circle_radius=2))

        # 4. Pose detection
        mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(255,0,0),thickness=2,circle_radius=2),
                              mp_drawing.DrawingSpec(color=(240,0,0),thickness=2,circle_radius=2))
        
        # Export coordinates
        try:
            # Extract pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in pose]).flatten())

            # Extract face landmarks
            face = results.face_landmarks.landmark
            face_row = list(np.array([[landmark.x,landmark.y,landmark.z,landmark.visibility] for landmark in face]).flatten())

            # # Concat rows
            row = pose_row+face_row

            # # Insert row
            # row.insert(0,class_name)

            # # Export to csv
            # with open('coords.csv',mode='a',newline='') as f:
            #     csv_writer = csv.writer(f,delimiter=",",quotechar="'",quoting=csv.QUOTE_MINIMAL)
            #     csv_writer.writerow(row)

            # Make detections
            X = pd.DataFrame([row])
            body_language_class = model.predict(X)[0]
            body_language_pred = model.predict_proba(X)[0]
            print(body_language_class,body_language_pred)

            # Grab ear coords
            coords = tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x,
                                                results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)
                                                ),[640,480]).astype(int))
            
            cv2.rectangle(image,
                         (coords[0],coords[1]+5),
                         (coords[0]+len(body_language_class)*20,coords[1]-30),
                         (245,117,16),-1)
            cv2.putText(image,body_language_class,coords,
                        cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            
            # Display class
            cv2.putText(image,"CLASS",
                        (95,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,body_language_class.split(" ")[0],
                        (90,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)
            
            # Display prob
            cv2.putText(image,"PROB",
                        (15,12),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,0),1,cv2.LINE_AA)
            cv2.putText(image,str(round(body_language_pred[np.argmax(body_language_pred)],2)),
                        (10,40),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

        except Exception as e:
            print(e)

        cv2.imshow("Holistic model detection",image)

        if cv2.waitKey(10) & 0xFF==ord('q'):
            break
cap.release()
cv2.destroyAllWindows()            

Happy [0.83 0.11 0.06]
Happy [0.87 0.1  0.03]
Happy [0.88 0.09 0.03]
Happy [0.86 0.11 0.03]
Happy [0.85 0.12 0.03]
Happy [0.85 0.11 0.04]
Happy [0.86 0.1  0.04]
Happy [0.86 0.1  0.04]
Happy [0.87 0.09 0.04]
Happy [0.88 0.08 0.04]
Happy [0.86 0.1  0.04]
Happy [0.89 0.08 0.03]
Happy [0.85 0.09 0.06]
Happy [0.82 0.09 0.09]
Happy [0.85 0.09 0.06]
Happy [0.87 0.09 0.04]
Happy [0.88 0.08 0.04]
Happy [0.87 0.09 0.04]
Happy [0.88 0.08 0.04]
Happy [0.88 0.08 0.04]
Happy [0.87 0.08 0.05]
Happy [0.87 0.08 0.05]
Happy [0.88 0.08 0.04]
Happy [0.86 0.09 0.05]
Happy [0.86 0.09 0.05]
Happy [0.87 0.09 0.04]
Happy [0.86 0.1  0.04]
Happy [0.86 0.1  0.04]
Happy [0.85 0.12 0.03]
Happy [0.85 0.12 0.03]
Happy [0.86 0.11 0.03]
Happy [0.85 0.11 0.04]
Happy [0.86 0.1  0.04]
Happy [0.85 0.11 0.04]
Happy [0.85 0.11 0.04]
Happy [0.84 0.11 0.05]
Happy [0.84 0.12 0.04]
Happy [0.86 0.1  0.04]
Happy [0.84 0.1  0.06]
Happy [0.83 0.11 0.06]
Happy [0.84 0.1  0.06]
Happy [0.81 0.11 0.08]
Happy [0.87 0.08 0.05]
Happy [0.86

In [83]:
results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y

0.5581431984901428

In [87]:
tuple(np.multiply(np.array((results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x,
         results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)),[640,480]).astype(int))

(453, 267)

In [58]:
# cap = cv2.VideoCapture(0)
# # Initiate holistic model
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
#     while cap.isOpened():
#         ret, frame = cap.read()
        
#         # Recolor Feed
#         image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         image.flags.writeable = False        
        
#         # Make Detections
#         results = holistic.process(image)
#         # print(results.face_landmarks)
        
#         # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
#         # Recolor image back to BGR for rendering
#         image.flags.writeable = True   
#         image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
#         # 1. Draw face landmarks
#         mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
#                                  mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
#                                  mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
#                                  )
        
#         # 2. Right hand
#         mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
#                                  mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
#                                  mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
#                                  )

#         # 3. Left Hand
#         mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
#                                  mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
#                                  mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
#                                  )

#         # 4. Pose Detections
#         mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
#                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
#                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2))
#         try:
#             # Extract Pose landmarks
#             pose = results.pose_landmarks.landmark
#             pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
            
#             # Extract Face landmarks
#             face = results.face_landmarks.landmark
#             face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
            
#             # Concate rows
#             row = pose_row+face_row
            
# #             # Append class name 
# #             row.insert(0, class_name)
            
# #             # Export to CSV
# #             with open('coords.csv', mode='a', newline='') as f:
# #                 csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
# #                 csv_writer.writerow(row) 

#             # Make Detections
#             X = pd.DataFrame([row])
#             body_language_class = model.predict(X)[0]
#             body_language_prob = model.predict_proba(X)[0]
#             print(body_language_class, body_language_prob)
#             # Grab ear coords
#             coords = tuple(np.multiply(
#                             np.array(
#                                 (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
#                                  results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
#                         , [640,480]).astype(int))
            
#             cv2.rectangle(image, 
#                           (coords[0], coords[1]+5), 
#                           (coords[0]+len(body_language_class)*20, coords[1]-30), 
#                           (245, 117, 16), -1)
#             cv2.putText(image, body_language_class, coords, 
#                         cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
#             # Get status box
#             cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
#             # Display Class
#             cv2.putText(image, 'CLASS'
#                         , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
#             cv2.putText(image, body_language_class.split(' ')[0]
#                         , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
#             # Display Probability
#             cv2.putText(image, 'PROB'
#                         , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
#             cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
#                         , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
#         except:
#             pass
                        
#         cv2.imshow('Raw Webcam Feed', image)

#         if cv2.waitKey(10) & 0xFF == ord('q'):
#             break

# cap.release()
# cv2.destroyAllWindows()

/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: 

Victorius [0.29 0.14 0.57]
Victorius [0.29 0.17 0.54]
Victorius [0.28 0.17 0.55]


/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: 

Victorius [0.28 0.17 0.55]
Victorius [0.27 0.17 0.56]
Victorius [0.28 0.17 0.55]


/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: 

...

/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: 

Happy [0.62 0.09 0.29]
Happy [0.61 0.09 0.3 ]
Happy [0.6  0.09 0.31]


/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: 

Happy [0.6  0.09 0.31]
Happy [0.6  0.09 0.31]
Happy [0.6  0.09 0.31]
Happy [0.61 0.1  0.29]
Happy [0.62 0.1  0.28]


/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/home/rhythm/.local/lib/python3.10/site-packages/sklearn/base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
